# Loyalty Points Analysis Notebook
This notebook computes loyalty points for players.

## 1. Environment Setup

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time

## 2. Parameters and File Paths

In [2]:
YEAR = 2023
MONTH = 10  # October
# File paths for the CSVs or data sources (update as needed)
DEPOSITS_FILE = "deposits_october.csv"
WITHDRAWALS_FILE = "withdrawals_october.csv"
GAMES_FILE = "games_october.csv"

 ## 3. Load Data

In [3]:
# Load datasets, parsing timestamps. Ensure column names match actual files.
# Example: adjust parse_dates column if timestamp column named differently
try:
    deposits = pd.read_csv(DEPOSITS_FILE, parse_dates=["timestamp"])
    withdrawals = pd.read_csv(WITHDRAWALS_FILE, parse_dates=["timestamp"])
    games = pd.read_csv(GAMES_FILE, parse_dates=["timestamp"])
except Exception as e:
    print(f"Error loading files: {e}")
    # If files not found or names differ, adjust DEPOSITS_FILE, etc.


# Inspect dataframes briefly

# %%
for df, name in [(deposits, 'deposits'), (withdrawals, 'withdrawals'), (games, 'games')]:
    print(f"{name} head:")
    display(df.head())
    print(df.info())


deposits head:


,player_id,deposit_amount,timestamp
0,player_001,337.51,2023-10-06 09:42:36
1,player_001,4337.57,2023-10-02 05:08:22
2,player_001,3025.52,2023-10-14 08:25:56
3,player_001,3554.96,2023-10-12 21:19:10
4,player_001,151.89,2023-10-11 20:03:33


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   player_id       508 non-null    object        
 1   deposit_amount  508 non-null    float64       
 2   timestamp       508 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 12.0+ KB
None
withdrawals head:


,player_id,withdrawal_amount,timestamp
0,player_001,432.07,2023-10-12 21:15:44
1,player_001,2671.81,2023-10-24 01:34:26
2,player_002,1254.94,2023-10-30 16:11:08
3,player_002,1149.43,2023-10-04 12:51:03
4,player_002,816.27,2023-10-23 02:35:53


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   player_id          313 non-null    object        
 1   withdrawal_amount  313 non-null    float64       
 2   timestamp          313 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 7.5+ KB
None
games head:


,player_id,game_id,timestamp
0,player_001,game_1,2023-10-11 22:56:40
1,player_001,game_2,2023-10-26 08:50:28
2,player_001,game_3,2023-10-19 11:10:21
3,player_001,game_4,2023-10-22 20:53:14
4,player_001,game_5,2023-10-22 11:48:26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1978 entries, 0 to 1977
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   player_id  1978 non-null   object        
 1   game_id    1978 non-null   object        
 2   timestamp  1978 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 46.5+ KB
None


## 4. Preprocessing: date, time, slot assignment

In [4]:
# - Extract date, time
# - Assign slot: S1 for 00:00-11:59:59, S2 for 12:00:00-23:59:59


def assign_slot(ts):
    """Assign slot S1 if time < 12:00:00, else S2."""
    t = ts.time()
    if t >= time(0, 0, 0) and t < time(12, 0, 0):
        return 'S1'
    else:
        return 'S2'

# Ensure timestamp columns exist and are datetime
for df, col in [(deposits, 'timestamp'), (withdrawals, 'timestamp'), (games, 'timestamp')]:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in dataframe")
    # If not datetime, convert
    if not np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = pd.to_datetime(df[col])

# Add date, time, slot, day
for df in [deposits, withdrawals, games]:
    df['date'] = df['timestamp'].dt.date
    df['time'] = df['timestamp'].dt.time
    df['slot'] = df['timestamp'].apply(assign_slot)
    df['day'] = df['timestamp'].dt.day


## 5. Define Loyalty Computation Function

In [5]:
def compute_loyalty(df_deposits, df_withdrawals, df_games):
    """
    Compute loyalty metrics and points for each player in the given dataframes.
    Returns a DataFrame with columns:
      player_id, total_deposit_amount, total_withdrawal_amount,
      count_deposits, count_withdrawals, diff_count, count_games, loyalty_points
    """
    # Aggregations for deposits
    if not df_deposits.empty:
        dep_agg = df_deposits.groupby('player_id').agg(
            total_deposit_amount=('deposit_amount', 'sum'),
            count_deposits=('deposit_amount', 'count')
        ).reset_index()
    else:
        dep_agg = pd.DataFrame(columns=['player_id', 'total_deposit_amount', 'count_deposits'])

    # Aggregations for withdrawals
    if not df_withdrawals.empty:
        wdraw_agg = df_withdrawals.groupby('player_id').agg(
            total_withdrawal_amount=('withdrawal_amount', 'sum'),
            count_withdrawals=('withdrawal_amount', 'count')
        ).reset_index()
    else:
        wdraw_agg = pd.DataFrame(columns=['player_id', 'total_withdrawal_amount', 'count_withdrawals'])

    # Aggregations for games
    if not df_games.empty:
        # Assuming games dataframe has a column 'game_id'; if not, count rows
        if 'game_id' in df_games.columns:
            games_agg = df_games.groupby('player_id').agg(
                count_games=('game_id', 'count')
            ).reset_index()
        else:
            games_agg = df_games.groupby('player_id').size().reset_index(name='count_games')
    else:
        games_agg = pd.DataFrame(columns=['player_id', 'count_games'])

    # Union of player_ids
    all_ids = pd.Series(
        pd.concat([
            dep_agg['player_id'] if not dep_agg.empty else pd.Series(dtype=object),
            wdraw_agg['player_id'] if not wdraw_agg.empty else pd.Series(dtype=object),
            games_agg['player_id'] if not games_agg.empty else pd.Series(dtype=object)
        ]).unique(), name='player_id'
    )
    players = pd.DataFrame(all_ids)

    # Merge
    df = players.merge(dep_agg, on='player_id', how='left') \
                .merge(wdraw_agg, on='player_id', how='left') \
                .merge(games_agg, on='player_id', how='left')
    # Fill NaNs
    for col in ['total_deposit_amount', 'count_deposits', 'total_withdrawal_amount', 'count_withdrawals', 'count_games']:
        if col in df.columns:
            df[col] = df[col].fillna(0)
        else:
            df[col] = 0
    # Compute diff_count
    df['diff_count'] = (df['count_deposits'] - df['count_withdrawals']).clip(lower=0)
    # Loyalty formula
    df['loyalty_points'] = (
        0.01 * df['total_deposit_amount']
        + 0.005 * df['total_withdrawal_amount']
        + 0.001 * df['diff_count']
        + 0.2 * df['count_games']
    )
    return df

## 6. Part A.1: Slot-wise Loyalty for Specific Dates

In [6]:
# Dates and slots of interest: 2 Oct S1, 16 Oct S2, 18 Oct S1, 26 Oct S2
# Adjust the year if needed.

# %%
dates_slots = [
    (f"{YEAR}-10-02", 'S1'),
    (f"{YEAR}-10-16", 'S2'),
    (f"{YEAR}-10-18", 'S1'),
    (f"{YEAR}-10-26", 'S2'),
]

# Dictionary to hold results
slot_results = {}
for date_str, slot in dates_slots:
    date_obj = pd.to_datetime(date_str).date()
    dep_filt = deposits[(deposits['date'] == date_obj) & (deposits['slot'] == slot)]
    wdraw_filt = withdrawals[(withdrawals['date'] == date_obj) & (withdrawals['slot'] == slot)]
    games_filt = games[(games['date'] == date_obj) & (games['slot'] == slot)]
    df_loyal = compute_loyalty(dep_filt, wdraw_filt, games_filt)
    df_loyal_sorted = df_loyal.sort_values('loyalty_points', ascending=False).reset_index(drop=True)
    slot_results[(date_str, slot)] = df_loyal_sorted
    print(f"\nLoyalty points for {date_str} slot {slot}, top rows:")
    display(df_loyal_sorted.head())


Loyalty points for 2023-10-02 slot S1, top rows:


,player_id,total_deposit_amount,count_deposits,total_withdrawal_amount,count_withdrawals,count_games,diff_count,loyalty_points
0,player_001,4337.57,1.0,0.0,0.0,0.0,1.0,43.3767
1,player_088,3341.93,1.0,0.0,0.0,0.0,1.0,33.4203
2,player_100,2985.05,1.0,0.0,0.0,0.0,1.0,29.8515
3,player_003,2982.45,1.0,0.0,0.0,0.0,1.0,29.8255
4,player_040,1787.03,1.0,0.0,0.0,0.0,1.0,17.8713



Loyalty points for 2023-10-16 slot S2, top rows:


,player_id,total_deposit_amount,count_deposits,total_withdrawal_amount,count_withdrawals,count_games,diff_count,loyalty_points
0,player_086,4922.79,1.0,0.00,0.0,1.0,1.0,49.42890
1,player_097,4939.23,1.0,0.00,0.0,0.0,1.0,49.39330
2,player_095,3496.63,1.0,0.00,0.0,1.0,1.0,35.16730
3,player_033,2022.60,1.0,0.00,0.0,0.0,1.0,20.22700
4,player_016,0.00,0.0,2542.11,1.0,1.0,0.0,12.91055



Loyalty points for 2023-10-18 slot S1, top rows:


,player_id,total_deposit_amount,count_deposits,total_withdrawal_amount,count_withdrawals,count_games,diff_count,loyalty_points
0,player_009,3815.89,1.0,0.0,0.0,0.0,1.0,38.1599
1,player_046,2939.10,1.0,0.0,0.0,2.0,1.0,29.7920
2,player_079,2845.96,1.0,0.0,0.0,0.0,1.0,28.4606
3,player_014,1429.30,1.0,0.0,0.0,1.0,1.0,14.4940
4,player_080,950.54,1.0,0.0,0.0,0.0,1.0,9.5064



Loyalty points for 2023-10-26 slot S2, top rows:


,player_id,total_deposit_amount,count_deposits,total_withdrawal_amount,count_withdrawals,count_games,diff_count,loyalty_points
0,player_037,4953.00,1.0,0.0,0.0,0.0,1.0,49.5310
1,player_026,4001.56,1.0,0.0,0.0,0.0,1.0,40.0166
2,player_056,2339.55,1.0,0.0,0.0,1.0,1.0,23.5965
3,player_068,1950.26,1.0,0.0,0.0,0.0,1.0,19.5036
4,player_036,1628.93,1.0,0.0,0.0,0.0,1.0,16.2903


 ## 7. Part A.2: Overall Loyalty Points and Ranking for October

In [7]:
# %%
# Define start and end timestamps for October
start_ts = pd.to_datetime(f"{YEAR}-10-01 00:00:00")
end_ts = pd.to_datetime(f"{YEAR}-10-31 23:59:59")

dep_oct = deposits[(deposits['timestamp'] >= start_ts) & (deposits['timestamp'] <= end_ts)]
wdraw_oct = withdrawals[(withdrawals['timestamp'] >= start_ts) & (withdrawals['timestamp'] <= end_ts)]
games_oct = games[(games['timestamp'] >= start_ts) & (games['timestamp'] <= end_ts)]

# Compute loyalty for full month
loyalty_oct = compute_loyalty(dep_oct, wdraw_oct, games_oct)
# Rank by loyalty_points desc, then count_games desc
loyalty_oct = loyalty_oct.sort_values(by=['loyalty_points', 'count_games'], ascending=[False, False]).reset_index(drop=True)
loyalty_oct['rank'] = np.arange(1, len(loyalty_oct) + 1)

print("Overall October loyalty, top 10:")
display(loyalty_oct.head(10))

# Extract top 50
top50 = loyalty_oct[loyalty_oct['rank'] <= 50].copy()
print("Top 50 players:")
display(top50)

Overall October loyalty, top 10:


,player_id,total_deposit_amount,count_deposits,total_withdrawal_amount,count_withdrawals,count_games,diff_count,loyalty_points,rank
0,player_026,29411.48,8,1646.82,1.0,24,7.0,307.15590,1
1,player_044,22131.11,7,12789.10,7.0,11,0.0,287.45660,2
2,player_074,25021.77,7,5849.50,3.0,7,4.0,280.86920,3
3,player_047,25577.84,9,3357.30,3.0,19,6.0,276.37090,4
4,player_094,26684.67,9,85.72,1.0,24,8.0,272.08330,5
5,player_066,25391.47,8,2411.05,1.0,24,7.0,270.77695,6
6,player_018,23823.11,7,1766.82,1.0,22,6.0,251.47120,7
7,player_068,20564.41,8,7236.42,4.0,11,4.0,244.03020,8
8,player_048,22918.01,7,2137.47,3.0,18,4.0,243.47145,9
9,player_086,21120.97,8,3670.69,4.0,20,4.0,233.56715,10


Top 50 players:


,player_id,total_deposit_amount,count_deposits,total_withdrawal_amount,count_withdrawals,count_games,diff_count,loyalty_points,rank
0,player_026,29411.48,8,1646.82,1.0,24,7.0,307.15590,1
1,player_044,22131.11,7,12789.10,7.0,11,0.0,287.45660,2
2,player_074,25021.77,7,5849.50,3.0,7,4.0,280.86920,3
3,player_047,25577.84,9,3357.30,3.0,19,6.0,276.37090,4
4,player_094,26684.67,9,85.72,1.0,24,8.0,272.08330,5
5,player_066,25391.47,8,2411.05,1.0,24,7.0,270.77695,6
6,player_018,23823.11,7,1766.82,1.0,22,6.0,251.47120,7
7,player_068,20564.41,8,7236.42,4.0,11,4.0,244.03020,8
8,player_048,22918.01,7,2137.47,3.0,18,4.0,243.47145,9
9,player_086,21120.97,8,3670.69,4.0,20,4.0,233.56715,10


## 8. Part A.3: Average Deposit Amount per Transaction in October

In [8]:
# %%
if not dep_oct.empty:
    avg_deposit_per_txn = dep_oct['deposit_amount'].mean()
    print(f"Average deposit amount per transaction in October: {avg_deposit_per_txn:.2f}")
else:
    print("No deposits in October data.")

# %% [markdown]

Average deposit amount per transaction in October: 2483.33


## 9. Part A.4: Average Deposit Amount per User in October

In [9]:
# Interpretation 1: Among users with ≥1 deposit

# %%
dep_sum_by_user = dep_oct.groupby('player_id')['deposit_amount'].sum().reset_index(name='total_deposit_by_user')
if not dep_sum_by_user.empty:
    avg_dep_per_user = dep_sum_by_user['total_deposit_by_user'].mean()
    print(f"Average total deposit per depositing user: {avg_dep_per_user:.2f}")
else:
    print("No depositing users in October.")

# Interpretation 2: Among all active users (any activity)
# Build active user set
active_ids = set(pd.concat([dep_oct['player_id'], wdraw_oct['player_id'], games_oct['player_id']]).unique())
df_active = pd.DataFrame({'player_id': list(active_ids)})
df_active = df_active.merge(dep_sum_by_user, on='player_id', how='left')
df_active['total_deposit_by_user'] = df_active['total_deposit_by_user'].fillna(0)
avg_dep_per_active = df_active['total_deposit_by_user'].mean()
print(f"Average total deposit per active user: {avg_dep_per_active:.2f}")

# %% [markdown]

Average total deposit per depositing user: 12615.34
Average total deposit per active user: 12615.34


## 10. Part A.5: Average Number of Games Played per User in October

In [10]:
# Interpretation 1: Among users who played ≥1 game

# %%
games_count_by_user = games_oct.groupby('player_id').size().reset_index(name='count_games')
if not games_count_by_user.empty:
    avg_games_per_gamer = games_count_by_user['count_games'].mean()
    print(f"Average number of games among users who played at least 1: {avg_games_per_gamer:.2f}")
else:
    print("No game records in October.")

# Interpretation 2: Among all active users
df_active2 = pd.DataFrame({'player_id': list(active_ids)})
df_active2 = df_active2.merge(games_count_by_user, on='player_id', how='left')
df_active2['count_games'] = df_active2['count_games'].fillna(0)
avg_games_per_active = df_active2['count_games'].mean()
print(f"Average number of games per active user: {avg_games_per_active:.2f}")

# %% [markdown]

Average number of games among users who played at least 1: 19.78
Average number of games per active user: 19.78


## 11. Part B: Bonus Allocation Strategy for Top 50

In [11]:
# Pool: Rs 50,000

# %%
total_pool = 50000
floor_each = 200  # e.g., floor for each of 50 players
total_floor = floor_each * len(top50) if not top50.empty else 0
remaining_pool = total_pool - total_floor

if not top50.empty:
    sum_points_top50 = top50['loyalty_points'].sum()
    if sum_points_top50 > 0 and remaining_pool > 0:
        top50['prop_share'] = top50['loyalty_points'] / sum_points_top50 * remaining_pool
    else:
        top50['prop_share'] = 0
    top50['bonus_before_cap'] = floor_each + top50['prop_share']
    # Optional cap example
    cap = 5000
    top50['bonus_capped'] = top50['bonus_before_cap'].apply(lambda x: min(x, cap))
    print("Bonus allocation for top 50 (before cap and after cap):")
    display(top50[['player_id', 'rank', 'loyalty_points', 'bonus_before_cap', 'bonus_capped']])
else:
    print("No top 50 data to allocate bonus.")

# %% [markdown]

Bonus allocation for top 50 (before cap and after cap):


,player_id,rank,loyalty_points,bonus_before_cap,bonus_capped
0,player_026,1,307.15590,1387.463089,1387.463089
1,player_044,2,287.45660,1311.305700,1311.305700
2,player_074,3,280.86920,1285.838846,1285.838846
3,player_047,4,276.37090,1268.448442,1268.448442
4,player_094,5,272.08330,1251.872603,1251.872603
5,player_066,6,270.77695,1246.822261,1246.822261
6,player_018,7,251.47120,1172.186333,1172.186333
7,player_068,8,244.03020,1143.419466,1143.419466
8,player_048,9,243.47145,1141.259342,1141.259342
9,player_086,10,233.56715,1102.969370,1102.969370


## 12. Part C: Instructions for Empirical Fairness Analysis

In [12]:
# (This section is a placeholder: run exploratory analyses to understand distribution of each component.)
# You may add cells here to:
# - Plot distribution of deposit_amounts, withdrawal_amounts, count_games
# - Compute contribution of each term to loyalty_points per user
# - Identify outliers in loyalty_points


# ### Example: contribution analysis
# Compute per-player breakdown of loyalty term contributions in October

df_contrib = loyalty_oct.copy()
# Add columns for each term
if 'total_deposit_amount' in df_contrib.columns:
    df_contrib['term_deposit'] = 0.01 * df_contrib['total_deposit_amount']
if 'total_withdrawal_amount' in df_contrib.columns:
    df_contrib['term_withdrawal'] = 0.005 * df_contrib['total_withdrawal_amount']
if 'diff_count' in df_contrib.columns:
    df_contrib['term_diff_count'] = 0.001 * df_contrib['diff_count']
if 'count_games' in df_contrib.columns:
    df_contrib['term_games'] = 0.2 * df_contrib['count_games']
# Display breakdown for top 10
print("Contribution breakdown for top 10 players:")
display(df_contrib[['player_id', 'rank', 'loyalty_points', 'term_deposit', 'term_withdrawal', 'term_diff_count', 'term_games']].head(10))


# add more charts or analyses here as per business questions.


Contribution breakdown for top 10 players:


,player_id,rank,loyalty_points,term_deposit,term_withdrawal,term_diff_count,term_games
0,player_026,1,307.15590,294.1148,8.23410,0.007,4.8
1,player_044,2,287.45660,221.3111,63.94550,0.000,2.2
2,player_074,3,280.86920,250.2177,29.24750,0.004,1.4
3,player_047,4,276.37090,255.7784,16.78650,0.006,3.8
4,player_094,5,272.08330,266.8467,0.42860,0.008,4.8
5,player_066,6,270.77695,253.9147,12.05525,0.007,4.8
6,player_018,7,251.47120,238.2311,8.83410,0.006,4.4
7,player_068,8,244.03020,205.6441,36.18210,0.004,2.2
8,player_048,9,243.47145,229.1801,10.68735,0.004,3.6
9,player_086,10,233.56715,211.2097,18.35345,0.004,4.0


## 13. Save Results

In [13]:
# Optionally, save results to CSV or Excel for reporting.

# %%
# Example: save loyalty_oct and top50
try:
    loyalty_oct.to_csv("loyalty_october_results.csv", index=False)
    top50.to_csv("top50_loyalty_october.csv", index=False)
    print("Results saved to CSV files.")
except Exception as e:
    print(f"Error saving results: {e}")

# %% [markdown]


Results saved to CSV files.


## End of Notebook
 Replace file paths, YEAR, month, and column names as needed. Once actual data is loaded, run all cells to compute results.